<a href="https://colab.research.google.com/github/truongnc160466/main/blob/main/ViT_Tomato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Cài đặt thư viện cần thiết
!pip install torch torchvision transformers datasets

In [11]:
!pip install datasets

In [12]:
import os
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

class TomatoDataset(Dataset):
    def __init__(self, img_dir, label_dir, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transform = transform
        # 6 lớp, trong YOLO thì các class index có thể từ 0 đến 5
        self.labels_map = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}
        self.data = []
        self.load_data()

    def load_data(self):
        # Lặp qua tất cả các file ảnh trong thư mục images
        for img_name in os.listdir(self.img_dir):
            if img_name.endswith(".jpg") or img_name.endswith(".png"):  # Kiểm tra file ảnh
                img_path = os.path.join(self.img_dir, img_name)
                # Tìm nhãn tương ứng trong thư mục labels
                label_name = img_name.replace('.jpg', '.txt').replace('.png', '.txt')
                label_path = os.path.join(self.label_dir, label_name)
                with open(label_path, 'r') as f:
                    # Lấy class index từ dòng đầu tiên, bỏ qua các tọa độ bounding box
                    label_str = f.readline().strip().split()[0]  # Chỉ lấy giá trị class index
                    label = int(label_str)  # Chuyển class index thành số nguyên
                    self.data.append((img_path, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        # Trả về từ điển với các khóa mà Trainer yêu cầu
        return {"pixel_values": image, "labels": label}

# Định nghĩa các phép biến đổi (transform) cho dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize ảnh về kích thước 224x224
    transforms.ToTensor(),  # Chuyển đổi ảnh thành tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Chuẩn hóa ảnh
])

# Tạo dataset và DataLoader cho train và test
train_dataset = TomatoDataset(
    img_dir='/content/Laboro_Tomato/train/images',
    label_dir='/content/Laboro_Tomato/train/labels',
    transform=transform
)

test_dataset = TomatoDataset(
    img_dir='/content/Laboro_Tomato/test/images',
    label_dir='/content/Laboro_Tomato/test/labels',
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
from transformers import ViTForImageClassification, ViTFeatureExtractor

# Tải mô hình Vision Transformer pre-trained từ Hugging Face
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224-in21k',
    num_labels=6  # Phân loại thành 6 lớp
)

# Tạo feature extractor để tiền xử lý ảnh cho mô hình ViT
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [14]:
from torch.utils.data import default_collate

def collate_fn(batch):
    return default_collate(batch)

In [16]:
import torch

# Chuẩn bị thiết bị GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [17]:
from transformers import Trainer, TrainingArguments

# Chuẩn bị thiết bị GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Định nghĩa hàm tính độ chính xác (accuracy)
def compute_metrics(p):
    pred = p.predictions.argmax(-1)
    return {"accuracy": (pred == p.label_ids).astype(float).mean().item()}

# Định nghĩa các tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
)

# Huấn luyện mô hình với Trainer của Hugging Face
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    data_collator=collate_fn  # Hàm collate_fn
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.444300,1.406643,0.484472
2,1.143700,1.188667,0.552795
3,0.904600,1.069209,0.590062
4,0.708200,1.056331,0.596273
5,0.558500,1.103010,0.577640
6,0.449600,1.105469,0.583851
7,0.352900,1.064837,0.614907
8,0.272800,1.059741,0.602484
9,0.240400,1.070295,0.596273
10,0.227500,1.069037,0.596273


TrainOutput(global_step=210, training_loss=0.6488709324882144, metrics={'train_runtime': 16859.0522, 'train_samples_per_second': 0.381, 'train_steps_per_second': 0.012, 'total_flos': 4.982913566926848e+17, 'train_loss': 0.6488709324882144, 'epoch': 10.0})

In [18]:
# Đánh giá mô hình
results = trainer.evaluate()
print(f"Accuracy: {results['eval_accuracy'] * 100:.2f}%")

Accuracy: 59.63%
